<a href="https://colab.research.google.com/github/Shreemirrah/forese-test/blob/main/Pegaus_XSum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers

In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
!pip install sentencepiece

In [5]:
tokenizer=PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [6]:
model=PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
text='''
Recent work pre-training Transformers with
self-supervised objectives on large text corpora
has shown great success when fine-tuned on
downstream NLP tasks including text summarization. However, pre-training objectives tailored for abstractive text summarization have
not been explored. Furthermore there is a
lack of systematic evaluation across diverse domains. In this work, we propose pre-training
large Transformer-based encoder-decoder models on massive text corpora with a new selfsupervised objective. In PEGASUS, important
sentences are removed/masked from an input document and are generated together as one output
sequence from the remaining sentences, similar
to an extractive summary. We evaluated our best
PEGASUS model on 12 downstream summarization tasks spanning news, science, stories, instructions, emails, patents, and legislative bills. Experiments demonstrate it achieves state-of-the-art performance on all 12 downstream datasets measured
by ROUGE scores. Our model also shows surprising performance on low-resource summarization,
surpassing previous state-of-the-art results on 6
datasets with only 1000 examples. Finally we
validated our results using human evaluation and
show that our model summaries achieve human
performance on multiple datasets

Text summarization aims at generating accurate and concise
summaries from input document(s). In contrast to extractive
summarization which merely copies informative fragments
from the input, abstractive summarization may generate
novel words. A good abstractive summary covers principal
information in the input and is linguistically fluent.
In abstractive summarization, sequence-to-sequence
(Sutskever et al., 2014) has become a dominant framework
using encoder-decoder architectures based on RNNs
(Chung et al., 2014; Hochreiter & Schmidhuber, 1997)
and more recently Transformers (Vaswani et al., 2017).
Most prior work on neural abstractive summarization
relied on large-scale, high-quality datasets of supervised
document-summary pairs (Hermann et al., 2015) and
achieved promising results (Rush et al., 2015; Nallapati
et al., 2016; See et al., 2017). In recent years, there has
been increased interest in collecting new summarization
datasets that have more abstractive summaries (Narayan
et al., 2018), have longer documents, (Cohan et al., 2018;
Sharma et al., 2019), utilize multiple documents (Fabbri
et al., 2019), and are sourced from diverse domains (Grusky
et al., 2018; Koupaee & Wang, 2018; Kim et al., 2019;
Kornilova & Eidelman, 2019; Zhang & Tetreault, 2019);


however, there has been little work on systematic evaluation
of models across these broad settings.
Contemporaneously, the adoption of Transformer models
(Vaswani et al., 2017) pre-trained using self-supervised objectives on large text corpora (Radford et al., 2018a; Devlin
et al., 2019) have improved performance on many NLP tasks
(Wang et al., 2018; Rajpurkar et al., 2016).
Recent work leveraging such pre-training for Transformerbased sequence-to-sequence models (Dong et al., 2019;
Song et al., 2019; Rothe et al., 2019; Lewis et al., 2019;
Raffel et al., 2019) has extended the success to text generation, including abstractive summarization.
In this work, we study pre-training objectives specifically
for abstractive text summarization and evaluate on 12 downstream datasets spanning news (Hermann et al., 2015;
Narayan et al., 2018; Grusky et al., 2018; Rush et al., 2015;
Fabbri et al., 2019), science (Cohan et al., 2018), short
stories (Kim et al., 2019), instructions (Koupaee & Wang,
2018), emails (Zhang & Tetreault, 2019), patents (Sharma
et al., 2019), and legislative bills (Kornilova & Eidelman,
2019). We find that masking whole sentences from a document and generating these gap-sentences from the rest of the
document works well as a pre-training objective for downstream summarization tasks. In particular, choosing putatively important sentences outperforms lead or randomly
selected ones. We hypothesize this objective is suitable for
abstractive summarization as it closely resembles the downstream task, encouraging whole-document understanding
and summary-like generation. We call this self-supervised
objective Gap Sentences Generation (GSG). Using GSG
to pre-train a Transformer encoder-decoder on large corpora of documents (Web and news articles) results in our
method, Pre-training with Extracted Gap-sentences for Abstractive SUmmarization Sequence-to-sequence models, or
PEGASUS.
With our best 568M parameter model trained on the recently introduced C4 (Raffel et al., 2019) corpus we equal
or exceed state-of-the-art on the 12 summarization tasks
we consider. We further push forward the state-of-the-art
using a newly collected text corpus comprised of news-like
articles we call HugeNews, including the highly competitive
XSum and CNN/DailyMail summarization datasets.
Large-scale document-summary datasets are rare and in
practice there is a mismatch between research datasets and
real-world use-cases where collecting summaries is expensive; the most common setting is that of low-resource summarization. We simulate this setting and show that our
model is able to adapt very quickly when fine-tuning with
small numbers of supervised pairs, obtaining state-of-the-art
results in 6 datasets with only 1000 examples.
Qualitatively we observed high quality outputs from our


best models and validated this in human evaluation studies.
We found that PEGASUS summaries are at least as good as
reference summaries for the datasets we assessed – XSum,
CNN/DailyMail, and Reddit TIFU – even at low-levels of
supervision.
To summarize our contributions:
• We propose a new self-supervised pre-training objective for abstractive summarization, gap-sentences generation, and study strategies for selecting those sentences.
• We evaluate the proposed pre-training objective on a
broad range of downstream summarization tasks, with
careful ablations to choose the best model settings,
which we use to train a 568M parameter PEGASUS
model that surpasses or is on-par with the state-of-theart on all 12 downstream datasets considered.
• We show how good abstractive summarization performance can be achieved across broad domains with
very little supervision by fine-tuning the PEGASUS
model and surpassing previous state-of-the-art results
on many tasks with as little as 1000 examples.
• We conducted human evaluation studies to validate our
experimental design and demonstrate human-level summarization performance on XSum, CNN/DailyMail,
and Reddit TIFU.

We propose a new pre-training objective, GSG, in this
work, but for comparison, we also evaluate BERT’s maskedlanguage model objective, in isolation and in conjunction
with GSG.
3.1 Gap Sentences Generation (GSG)
We hypothesize that using a pre-training objective that more
closely resembles the downstream task leads to better and
faster fine-tuning performance. Given our intended use for
abstractive summarization, our proposed pre-training objective involves generating summary-like text from an input
document. In order to leverage massive text corpora for pretraining, we design a sequence-to-sequence self-supervised
objective in the absence of abstactive summaries. A naive
option would be to pre-train as an extractive summarizer;

however, such a procedure would only train a model to copy
sentences, thus not suitable for abstractive summarization.
Inspired by recent success in masking words and contiguous
spans (Joshi et al., 2019; Raffel et al., 2019), we select and
mask whole sentences from documents, and concatenate the
gap-sentences into a pseudo-summary. The corresponding
position of each selected gap sentence is replaced by a mask
token [MASK1] to inform the model. Gap sentences ratio,
or GSR, refers to the number of selected gap sentences to
the total number of sentences in the document, which is
similar to mask rate in other works.
To even more closely approximate a summary, we select
sentences that appear to be important/principal to the document. The resulting objective has both the empirically
demonstrated benefits of masking, and anticipates the form
of the downstream task.
We consider 3 primary strategies for selecting m gap sentences without replacement from a document, D = {xi}n,
comprised of n sentences:
Random Uniformly select m sentences at random.
Lead Select the first m sentences.
Principal Select top-m scored sentences according to importance. As a proxy for importance we compute ROUGE1-
F1 (Lin, 2004) between the sentence and the rest of the
document, si = rouge(xi
, D \ {xi}), ∀i.
In this formulation sentences are scored independently (Ind)
and the top m selected. We also consider selecting them
sequentially (Seq) as in Nallapati et al. (2017) by greedily
maximizing the ROUGE1-F1 between selected sentences,
S ∪ {xi}, and remaining sentences, D \ (S ∪ {xi}) as in
Algorithm 1.

3.2 Masked Language Model (MLM)
Following BERT, we select 15% tokens in the input text, and
the selected tokens are (1) 80% of time replaced by a mask
token [MASK2], or (2) 10% of time replaced by a random
token, or (3) 10% of time unchanged. We apply MLM to
train the Transformer encoder as the sole pre-training objective or along with GSG. When MLM is the sole pre-training
objective, the Transformer decoder shares all parameters
with encoder when fine-tuning on downstream tasks following Rothe et al. (2019).
Figure 1 simultaneously shows how both GSG and MLM
are applied to the same example when used in conjunction.
However, we found that MLM does not improve downstream tasks at large number of pre-training steps (section
6.1.2), and chose not to include MLM in the final model
PEGASUSLARGE (section 6.2).
4 Pre-training Corpus
For pre-training we considered two large text corpora:
• C4, or the Colossal and Cleaned version of Common
Crawl, introduced in Raffel et al. (2019); consists of
text from 350M Web-pages (750GB).
• HugeNews, a dataset of 1.5B articles (3.8TB) collected from news and news-like websites from 2013-
2019. A whitelist of domains ranging from highquality news publishers to lower-quality sites such as
high-school newspapers, and blogs was curated and
used to seed a web-crawler. Heuristics were used to
identify news-like articles, and only the main article
text was extracted as plain text.

5 Downstream Tasks/Datasets
For downstream summarization, we only used public abstractive summarization datasets, and access them through
TensorFlow Summarization Datasets 1
, which provides
publicly reproducible code for dataset processing and
train/validation/test splits. We used train/validation/test ratio of 80/10/10 if no split was provided, and 10% train split
as validation if there was no validation split.
XSum (Narayan et al., 2018) consists of 227k BBC articles
from 2010 to 2017 covering a wide variety of subjects along
with professionally written single-sentence summaries.
CNN/DailyMail (Hermann et al., 2015) dataset contains
93k articles from the CNN, and 220k articles the Daily Mail
newspapers. Both publishers supplement their articles with
bullet point summaries. We use the non-anonymized variant
used in See et al. (2017).
NEWSROOM (Grusky et al., 2018) is a large dataset containing 1.3M article-summary pairs written by authors and
editors in the newsrooms of 38 major publications between
1998 and 2017.
Multi-News (Fabbri et al., 2019) is a multi-document summarization dataset consisting of 56k pairs of news articles and their human-written summaries from the site
newser.com.
Gigaword (Rush et al., 2015) contains 4M examples extracted from news articles (seven publishers) from the Gigaword corpus (Graff et al., 2003). The task is to generate the
headline from the first sentence.
arXiv, PubMed (Cohan et al., 2018) are two long document
datasets of scientific publications from arXiv.org (113k) and
PubMed (215k). The task is to generate the abstract from
the paper body.
BIGPATENT (Sharma et al., 2019) consists of 1.3 million
U.S. patents along with human summaries under nine patent
classification categories.
WikiHow (Koupaee & Wang, 2018) is a large-scale dataset
of instructions from the online WikiHow.com website. Each
of 200k examples consists of multiple instruction-step paragraphs along with a summarizing sentence. The task is
to generate the concatenated summary-sentences from the
paragraphs.
Reddit TIFU (Kim et al., 2019) contains 120K posts of
informal stories from the online discussion forum Reddit,
more specifically the TIFU sub-reddit from 2013-Jan to
2018-Mar. The sub-reddit posts strictly follow the rule of
writing a descriptive ”TL;DR” summary and has higher quality

ity than (Volske et al., 2017) (which used more subreddits) ¨
based on our manual inspection. We uses the TIFU-long
subset (using TLDR as summaries) in the work.
AESLC (Zhang & Tetreault, 2019) consists of 18k email
bodies and their subjects from the Enron corpus (Klimt &
Yang, 2004), a collection of email messages of employees
in the Enron Corporation.
BillSum (Kornilova & Eidelman, 2019) contains 23k US
Congressional bills and human-written reference summaries
from the 103rd-115th (1993-2018) sessions of Congress. We
do not use the California test set which is out-of-distribution.
Following Grusky et al., the number of examples and extractive fragment coverage/density for all downstream datasets
is illustrated in Appendix A.
6 Experiments
In a similar strategy to Raffel et al. (2019), to save time
and computation we conducted pre-training ablation experiments using a reduced-size model with 223M parameters, PEGASUSBASE, smaller batch size, and only 4 of
12 datasets before scaling up pre-training with the best settings to the final 568M parameters, PEGASUSLARGE. The
datasets (XSum, CNN/DailyMail, WikiHow and Reddit
TIFU) were chosen for diversity in abstractiveness, writing
style, and size.
PEGASUSBASE had L = 12, H = 768, F = 3072, A = 12
and PEGASUSLARGE had L = 16, H = 1024, F =
4096, A = 16, where L denotes the number of layers for encoder and decoder (i.e. Transformer blocks),
H for the hidden size, F for the feed-forward layer
size and A for the number of self-attention heads. We
pre-trained PEGASUSBASE with a batch size of 256 and
PEGASUSLARGE with a batch size of 8192. We refer to
PEGASUSBASE without pre-training as TransformerBASE.
We used sinusoidal positional encoding following Vaswani
et al. (2017). For optimization, both pre-training and finetuning used Adafactor (Shazeer & Stern, 2018) with square
root learning rate decay and dropout rate of 0.1.
We used greedy-decoding for studies in Section 6.1, and
used beam-search with a length-penalty, α, as in Wu et al.
(2016) for the final large model.
All experiments’ hyper parameters can be found in Appendix C and reported numbers are in Appendix D.
'''

In [21]:
tokens=tokenizer(text,truncation=True,padding="longest",return_tensors="pt")

In [22]:
tokens

{'input_ids': tensor([[13618,   201,  1133,   121, 18006, 38979,   122,   813,   121, 83465,
          4358,   124,   423,  1352,   110, 88758,   148,  1673,   255,   924,
           173,  1226,   121, 37126,   124, 18030, 36789,  2722,   330,  1352,
          5906,  6520,  3884,   107,   611,   108,  1133,   121, 18006,  4358,
          5516,   118,  7093,  5551,  1352,  5906,  6520,  3884,   133,   146,
           174,  8678,   107,  5689,   186,   117,   114,  1905,   113, 11624,
          4051,   482,  2766,  9982,   107,   222,   136,   201,   108,   145,
         10287,  1133,   121, 18006,   423, 51979,   121,   936, 40753,   121,
          2534, 56636,  1581,   124,  2926,  1352,   110, 88758,   122,   114,
           177,   813, 83465,  4129,   107,   222, 49921, 89637,   108,   356,
          9750,   127,  2515,   191, 33238,   316,   135,   142,  3196,  2199,
           111,   127,  3943,   424,   130,   156,  2940,  5936,   135,   109,
          2756,  9750,   108,   984,  

In [23]:
summary=model.generate(**tokens)

In [24]:
{**tokens}

{'input_ids': tensor([[13618,   201,  1133,   121, 18006, 38979,   122,   813,   121, 83465,
           4358,   124,   423,  1352,   110, 88758,   148,  1673,   255,   924,
            173,  1226,   121, 37126,   124, 18030, 36789,  2722,   330,  1352,
           5906,  6520,  3884,   107,   611,   108,  1133,   121, 18006,  4358,
           5516,   118,  7093,  5551,  1352,  5906,  6520,  3884,   133,   146,
            174,  8678,   107,  5689,   186,   117,   114,  1905,   113, 11624,
           4051,   482,  2766,  9982,   107,   222,   136,   201,   108,   145,
          10287,  1133,   121, 18006,   423, 51979,   121,   936, 40753,   121,
           2534, 56636,  1581,   124,  2926,  1352,   110, 88758,   122,   114,
            177,   813, 83465,  4129,   107,   222, 49921, 89637,   108,   356,
           9750,   127,  2515,   191, 33238,   316,   135,   142,  3196,  2199,
            111,   127,  3943,   424,   130,   156,  2940,  5936,   135,   109,
           2756,  9750,   1

In [25]:
summary

tensor([[    0,  3414,   121, 18006, 51979,   121,   936, 40753,   121,  2534,
         56636,  1581,   118,  7093,  5551,  1352,  5906,  6520,  3884, 28336,
           449,   121,  1313,   121,   544,   121,  3904,   637,   124,   149,
           665, 18030, 24089,  5844,   141,   840, 34823, 13601,  4198,   107,
             1]])

In [26]:
tokenizer.decode(summary[0])

'<pad>Pre-training Transformer-based encoder-decoder models for abstractive text summarization achieves state-of-the-art performance on all 12 downstream datasets measured by ROUGE scores.</s>'